In [32]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'insurance:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F13720%2F18513%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240405%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240405T143503Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3Dc687aeb56c1d7211238fa0c340af148a7c1cbd2ac8b6512f989e9c9493337a0a7be3ee9e3e9a9b92c094778534c421b5e478adffb473ee91fe22ef47fd56ad06254ae0dd828f774c61520b1d6bd87beb3445dce5e5301f4b29fcaf1b57dbe4330f9d6ee4b7bb2da6b03086182c61762f994ad2c080ca8e9d77ea0133af16ec3dcada710f02b0adab910eab4e4143797d44745d0a8586fea500a7bdaacc6e7f9cda2ce16792bffc6a8bdece052c1138c54aadeec9ed8ffbdf83e7189a523e5697a5e4fa21ee4b98572b4fc6dca90adcbd27d66118f2319d6a2b8ec43435adf8df270e6c233e662c7b96d5b8d65fad434b55e13366037d1657a89a21becf5587fb'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


[==================================================] 16425 bytes downloaded
Downloaded and uncompressed: insurance
Data source import complete.


In [33]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/insurance/insurance.csv


In [34]:
df = pd.read_csv("/kaggle/input/insurance/insurance.csv")
df.head(3)

,age,sex,bmi,children,smoker,region,charges
0,19,female,27.90,0,yes,southwest,16884.9240
1,18,male,33.77,1,no,southeast,1725.5523
2,28,male,33.00,3,no,southeast,4449.4620


In [35]:
df['sex'].value_counts()

sex
male      676
female    662
Name: count, dtype: int64

In [36]:
df = pd.get_dummies(df,columns=['sex','region','smoker'] ,dtype='int')
df.head()

,age,bmi,children,charges,sex_female,sex_male,region_northeast,region_northwest,region_southeast,region_southwest,smoker_no,smoker_yes
0,19,27.900,0,16884.92400,1,0,0,0,0,1,0,1
1,18,33.770,1,1725.55230,0,1,0,0,1,0,1,0
2,28,33.000,3,4449.46200,0,1,0,0,1,0,1,0
3,33,22.705,0,21984.47061,0,1,0,1,0,0,1,0
4,32,28.880,0,3866.85520,0,1,0,1,0,0,1,0


In [37]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

df.iloc[:,0:12] = scaler.fit_transform(df.iloc[:,0:12])

In [38]:
from sklearn.decomposition import PCA
pca = PCA(n_components=2)  # Reduce to 3 dimensions for visualization
X_pca = pca.fit_transform(df.iloc[:,0:12])

In [39]:
x1_range = np.linspace(X_pca[:, 0].min(), X_pca[:, 0].max(), 10)
x2_range = np.linspace(X_pca[:, 1].min(), X_pca[:, 1].max(), 10)
x1_mesh, x2_mesh = np.meshgrid(x1_range, x2_range)
X_mesh = np.column_stack((x1_mesh.ravel(), x2_mesh.ravel()))

In [41]:
first_column = df.pop('charges')
df.insert(0, 'charges', first_column)

In [42]:
x = df.iloc[:,1:12]
y = df.iloc[:,0]

In [43]:
model = LinearRegression()
model.fit(X_pca, y)

LinearRegression()

In [44]:
y_pred_mesh = model.predict(X_mesh).reshape(x1_mesh.shape)

In [45]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(x , y ,test_size=0.2,random_state=2)

In [46]:
from sklearn.linear_model import LinearRegression

In [47]:
lr = LinearRegression()

In [48]:
lr.fit(X_train,y_train)

LinearRegression()

In [49]:
y_pred = lr.predict(X_test)

In [50]:
accuracy_score = lr.score(X_test,y_pred)
accuracy_score*100

100.0

In [51]:
from sklearn.datasets import make_regression

In [52]:
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score

In [53]:
print("MAE",mean_absolute_error(y_test,y_pred))
print("MSE",mean_squared_error(y_test,y_pred))
print("R2 score",r2_score(y_test,y_pred))

MAE 0.353547242663924
MSE 0.2613903931612512
R2 score 0.7449415564623687


In [54]:
import plotly.graph_objects as go

In [55]:
fig = go.Figure(data=[go.Scatter3d(
    x=X_pca[:, 0],
    y=X_pca[:, 1],
    z=y,
    mode='markers',
    marker=dict(size=5, color='blue'),
    name='Original Data (PCA)'
)])

# Add the plane of the regression model
fig.add_trace(go.Surface(
    x=x1_mesh,
    y=x2_mesh,
    z=y_pred_mesh,
    opacity=0.8,
    colorscale='Viridis',
    name='Regression Plane (PCA)'
))

# Set the layout
fig.update_layout(scene=dict(
    xaxis_title='Principal Component 1',
    yaxis_title='Principal Component 2',
    zaxis_title='Y'
))

# Show the plot
fig.show()